In [3]:
import pandas as pd
import numpy as np
import re
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

In [4]:
songs = pd.read_csv("spotify_songs.csv", header = 0)
songs[songs.track_artist == "Ed Sheeran"]

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
5,7fvUMiyapMsRRxr07cU8Ef,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67,2yiy9cd2QktrNvWC2EUi0k,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,8,-5.385,1,0.1270,0.0799,0.000000,0.1430,0.585,124.982,163049
10,3eFJI470ALf1EYjywPRsJy,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,58,49bXZtlI5kQSqQMSCnSaWO,Cross Me (feat. Chance the Rapper & PnB Rock) ...,2019-06-21,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-6.500,1,0.1810,0.1460,0.000005,0.1240,0.752,121.984,207894
15,1EJgymgJHcjSOGSHcYaxvW,South of the Border (feat. Camila Cabello & Ca...,Ed Sheeran,66,1imXAvLuowvPYIsFMYUPY0,South of the Border (feat. Camila Cabello & Ca...,2019-12-11,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,-4.603,1,0.0896,0.1300,0.000005,0.3650,0.722,125.028,188230
78,3vQhEd4lOg4mAV6CHCgoHW,South of the Border (feat. Camila Cabello & Ca...,Ed Sheeran,65,05Bru0ZVTxp4orAyrZIA77,South of the Border (feat. Camila Cabello & Ca...,2019-10-04,Dance Pop,37i9dQZF1DWZQaaqNMbbXa,pop,...,5,-4.388,1,0.0404,0.2200,0.000046,0.3350,0.491,110.012,173455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30270,4vUmTMuQqjdnvlZmAH61Qk,South of the Border (feat. Camila Cabello & Ca...,Ed Sheeran,91,3oIFxDIo2fwuk4lwCmFZCx,No.6 Collaborations Project,2019-07-12,Charts 2020 🔥Top 2020🔥Hits 2020🔥Summer 2020🔥Po...,3xMQTDLOIGvj3lWH5e5x6F,edm,...,9,-6.376,0,0.0824,0.1480,0.000000,0.0865,0.668,97.986,204467
30406,7qiZfU4dY1lWllzX7mPBI3,Shape of You,Ed Sheeran,86,3T4tUhGYeRNVUGevb0wThu,÷ (Deluxe),2017-03-03,2010 - 2011 - 2012 - 2013 - 2014 - 2015 - 2016...,2DjIfVDXGYDgRxw7IJTKVb,edm,...,1,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713
30428,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,90,5Nux7ozBJ5KJ02QYWwrneR,I Don't Care (with Justin Bieber),2019-05-10,2010 - 2011 - 2012 - 2013 - 2014 - 2015 - 2016...,2DjIfVDXGYDgRxw7IJTKVb,edm,...,6,-5.041,1,0.0442,0.0912,0.000000,0.0894,0.842,101.956,219947
30456,34gCuhDGsG4bRPIf9bb02f,Thinking out Loud,Ed Sheeran,83,1xn54DMo2qIqBuMqHtUsFd,x (Deluxe Edition),2014-06-21,2010 - 2011 - 2012 - 2013 - 2014 - 2015 - 2016...,2DjIfVDXGYDgRxw7IJTKVb,edm,...,2,-6.061,1,0.0295,0.4740,0.000000,0.1840,0.591,78.998,281560


In [5]:
# to check amount of data we are working with
songs.shape

(32833, 23)

In [6]:
# selecting required columns for analysis

songs_df = songs[['track_id', 'track_name', 'track_artist', 'track_popularity','track_album_name','playlist_name', 'playlist_genre']]

songs_df.head()


,track_id,track_name,track_artist,track_popularity,track_album_name,playlist_name,playlist_genre
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,pop
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),Pop Remix,pop
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),Pop Remix,pop
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,Pop Remix,pop
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),Pop Remix,pop


In [7]:
# to check for null vaules
songs_df.isnull().sum()

,0
track_id,0
track_name,5
track_artist,5
track_popularity,0
track_album_name,5
playlist_name,0
playlist_genre,0


In [8]:
# seeing the rows with null values

null_rows = songs_df[songs_df.isnull().any(axis=1)]
print(null_rows)

                     track_id track_name track_artist  track_popularity  \
8151   69gRFGOWY9OMpFJgFol1u0        NaN          NaN                 0   
9282   5cjecvX0CmC9gK0Laf5EMQ        NaN          NaN                 0   
9283   5TTzhRSWQS4Yu8xTgAuq6D        NaN          NaN                 0   
19568  3VKFip3OdAvv4OfNTgFWeQ        NaN          NaN                 0   
19811  69gRFGOWY9OMpFJgFol1u0        NaN          NaN                 0   

      track_album_name       playlist_name playlist_genre  
8151               NaN             HIP&HOP            rap  
9282               NaN         GANGSTA Rap            rap  
9283               NaN         GANGSTA Rap            rap  
19568              NaN  Reggaeton viejito🔥          latin  
19811              NaN       latin hip hop          latin  


In [9]:
# as we have rows with track_naem, artsist and album name empty it doesn;t make sense to populate these fields ourselves, thus, we drop them
songs_df.dropna(inplace=True)

<ipython-input-9-0499bfbde9b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs_df.dropna(inplace=True)


In [10]:
# to check if the null vaules were dropped correctly
songs_df.isnull().sum()

,0
track_id,0
track_name,0
track_artist,0
track_popularity,0
track_album_name,0
playlist_name,0
playlist_genre,0


In [11]:
# check for duplicated values

songs_df.duplicated().sum()

323

In [12]:
# we check for refrence/example a song name that has multiple entries
songs_df[songs_df["track_name"] == "Welcome To The Jungle"]

,track_id,track_name,track_artist,track_popularity,track_album_name,playlist_name,playlist_genre
3627,0bVtevEgtDIeRjCJbK3Lmv,Welcome To The Jungle,Guns N' Roses,78,Appetite For Destruction,80's Songs | Top 💯 80s Music Hits,pop
11255,0bVtevEgtDIeRjCJbK3Lmv,Welcome To The Jungle,Guns N' Roses,78,Appetite For Destruction,This Is Guns N' Roses,rock
12366,0eFvoRSTTaR2q8bSWVjwfp,Welcome To The Jungle,Guns N' Roses,14,Greatest Hits,Rock Classics,rock
12594,5NIPsWpDjJTFBoPxCUUeXp,Welcome To The Jungle,Guns N' Roses,19,Appetite For Destruction,Classic Rock Workout,rock
12901,0oWHLtxWeMJhmwxtrxhNK0,Welcome To The Jungle,Guns N' Roses,1,Appetite For Destruction (Explicit Version),"Classic Rock 70s 80s 90s, Rock Classics - 70s ...",rock
13263,0bVtevEgtDIeRjCJbK3Lmv,Welcome To The Jungle,Guns N' Roses,78,Appetite For Destruction,Workday: Rock Classics,rock
14927,5NIPsWpDjJTFBoPxCUUeXp,Welcome To The Jungle,Guns N' Roses,19,Appetite For Destruction,’80s Hard Rock,rock
15158,0oWHLtxWeMJhmwxtrxhNK0,Welcome To The Jungle,Guns N' Roses,1,Appetite For Destruction (Explicit Version),"Classic Rock 70s 80s 90s, Rock Classics - 70s ...",rock
15677,1dM25yro2mjIAO5D1SvFDr,Welcome To The Jungle,Guns N' Roses,0,Greatest Hits (EU Version),Classic Hard Rock,rock
16155,0bVtevEgtDIeRjCJbK3Lmv,Welcome To The Jungle,Guns N' Roses,78,Appetite For Destruction,Hard Rock Classics 1967-1991 (Party Edition),rock


In [13]:
grouped_df = songs_df.groupby(['track_name'], as_index=False).agg({'track_popularity': 'mean',
                                                                   'playlist_genre': lambda x: x.mode()[0]})

# Display the result
grouped_df.head()

,track_name,track_popularity,playlist_genre
0,"""I TRIED FOR YEARS... NOBODY LISTENED""",18.0,rap
1,"""This Is Seagull….""",34.0,pop
2,#1 Stunna,24.0,rap
3,#NAKAMA,26.0,r&b
4,#Natural,50.0,latin


In [14]:
# check that the example rows have been aggregated correctly
grouped_df[grouped_df["track_name"] == "Welcome To The Jungle"]

,track_name,track_popularity,playlist_genre
21739,Welcome To The Jungle,36.6,rock


In [15]:
# dropping the first value of the duplicates in original dataset as we want to keep
# first values for columns that don't matter

songs_unique = songs_df.drop_duplicates(subset = 'track_name', keep = 'first')

# we drop repeated columns in grouped_df to ease merge
songs_unique = songs_unique.drop(columns=['playlist_genre', 'track_popularity'])
songs_unique.head()

,track_id,track_name,track_artist,track_album_name,playlist_name
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),Pop Remix
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),Pop Remix
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,Pop Remix
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,Someone You Loved (Future Humans Remix),Pop Remix


In [16]:
# merged finla data frame
merged_df = pd.merge(songs_unique, grouped_df, on='track_name', how='inner')
merged_df.head()

,track_id,track_name,track_artist,track_album_name,playlist_name,track_popularity,playlist_genre
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,66.0,edm
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),Pop Remix,67.0,edm
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),Pop Remix,70.0,edm
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,Pop Remix,60.0,pop
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,Someone You Loved (Future Humans Remix),Pop Remix,69.0,pop


In [17]:
# removing unnecessary brackets to reduce noise before vectorization


def clean_text(text):
    # Remove parentheses, curly braces, and other brackets
    text = re.sub(r'[\(\)\{\}\[\]]', '', text)
    return text

# Apply this function to columns
merged_df['track_name'] = merged_df['track_name'].apply(clean_text)
merged_df['track_album_name'] = merged_df['track_album_name'].apply(clean_text)

In [18]:
combined_attr = merged_df['track_artist'] + merged_df['playlist_name'] + merged_df['track_album_name'] + merged_df['playlist_genre']

In [19]:
combined_attr.head()

,0
0,Ed SheeranPop RemixI Don't Care with Justin Bi...
1,Maroon 5Pop RemixMemories Dillon Francis Remixedm
2,Zara LarssonPop RemixAll the Time Don Diablo R...
3,The ChainsmokersPop RemixCall You Mine - The R...
4,Lewis CapaldiPop RemixSomeone You Loved Future...


In [20]:
vectorizer = TfidfVectorizer()

vectors = vectorizer.fit_transform(combined_attr)
print(vectors)

  (0, 32426)	0.19077823479647119
  (0, 23049)	0.36402578101585076
  (0, 22771)	0.31430102672233723
  (0, 4784)	0.33794942697224306
  (0, 20383)	0.26900729692729397
  (0, 43361)	0.19202120159581137
  (0, 6707)	0.3429729082294274
  (0, 11149)	0.22081827532047546
  (0, 32623)	0.3901021350594583
  (0, 35694)	0.32955471599337616
  (0, 12025)	0.2857967188850279
  (1, 14412)	0.43149254327893183
  (1, 10769)	0.40129835576475065
  (1, 32632)	0.49808092264992126
  (1, 1405)	0.45406839790815684
  (1, 23770)	0.3730649859051974
  (1, 32426)	0.2435849247389188
  (2, 10654)	0.41794322047294463
  (2, 39305)	0.3136462949217849
  (2, 38977)	0.13888456502993105
  (2, 32410)	0.45101340535658957
  (2, 21679)	0.4616596269737543
  (2, 44308)	0.39262684487586674
  (2, 32426)	0.24194607443755523
  (2, 11149)	0.28004302972431394
  :	:
  (23445, 34940)	0.4695364005337744
  (23445, 2634)	0.20229922034893266
  (23445, 12072)	0.21473305503362053
  (23445, 22798)	0.2395621993694479
  (23446, 1140)	0.5061889146085384

In [21]:
#standardization of numerical value
scaler = StandardScaler()
popularity_scaled = scaler.fit_transform(merged_df[['track_popularity']])

combined_features = hstack([vectors, popularity_scaled])

# creating a similarity matrix
similarity = cosine_similarity(combined_features)
print(similarity)

[[ 1.          0.61223249  0.6578195  ... -0.57373693 -0.56354767
  -0.37104102]
 [ 0.61223249  1.          0.64797329 ... -0.58242112 -0.57207763
  -0.37665717]
 [ 0.6578195   0.64797329  1.         ... -0.60532081 -0.59457063
  -0.39146661]
 ...
 [-0.57373693 -0.58242112 -0.60532081 ...  1.          0.60206022
   0.43000917]
 [-0.56354767 -0.57207763 -0.59457063 ...  0.60206022  1.
   0.42404142]
 [-0.37104102 -0.37665717 -0.39146661 ...  0.43000917  0.42404142
   1.        ]]


In [22]:
# taking song name from user
song_name = input("Enter song name: ")

Enter song name: shape of you


In [23]:
# listing all the track names we have

lits_of_all_songs = merged_df['track_name'].tolist()

In [24]:
# finding closest match to the given input

find_close_match = difflib.get_close_matches(song_name, lits_of_all_songs)
close_match = find_close_match[0]
print(find_close_match)

['Shape of You', 'Isle of You', 'Ask of You']


In [25]:
index_of_song = merged_df[merged_df.track_name == close_match].index[0]
print(index_of_song)

1161


In [26]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_song]))
print(similarity_score)

[(0, 0.7096094730115028), (1, 0.6440661140436689), (2, 0.6693895661255223), (3, 0.5612743632016617), (4, 0.6614879526527849), (5, 0.7266049355025664), (6, 0.5889942542609), (7, 0.6614879526527848), (8, 0.6530601783788522), (9, 0.6440661140436689), (10, 0.620513047933733), (11, 0.6440661140436689), (12, 0.6440661140436689), (13, 0.6530601783788522), (14, -0.3859378451075617), (15, 0.7052311830217766), (16, 0.5612743632016618), (17, 0.6242042234874677), (18, 0.6614879526527849), (19, -0.16112709912463694), (20, 0.6693895661255224), (21, 0.6132416089368934), (22, 0.5889942542609), (23, 0.6693895661255224), (24, 0.5296011485268352), (25, 0.6015231442544947), (26, 0.6344627687853027), (27, -0.6758411133341952), (28, 0.5889942542609), (29, -0.3221007860573351), (30, 0.49347891146849826), (31, 0.49347891146849826), (32, 0.5296011485268352), (33, 0.5612743632016616), (34, 0.5612743632016618), (35, 0.47359834630358943), (36, 0.5459628681666339), (37, 0.6015231442544946), (38, 0.6440661140436688

In [27]:
len(similarity_score)

23449

In [28]:
sorted_similar_songs = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_songs)

[(1161, 0.9999999999999999), (1196, 0.8886078917737623), (1163, 0.8865701014764824), (1203, 0.8865701014764824), (1205, 0.8861290749572002), (1201, 0.882964427722507), (1189, 0.8827221984578718), (1169, 0.8827185179971839), (1158, 0.8792131928370985), (1197, 0.8786376329623243), (1183, 0.8784087355292602), (1177, 0.8779388665600502), (1195, 0.8772067927161973), (1209, 0.8749103058913321), (1171, 0.8743383439072743), (1191, 0.8716968929803819), (1180, 0.8714662974412415), (1170, 0.8693125898960344), (1204, 0.8664013309669073), (1228, 0.8660210505512729), (1207, 0.865497837670046), (1200, 0.8646487690159341), (1172, 0.8592454973257032), (1153, 0.8590718823319374), (1227, 0.8559020939619136), (1173, 0.8542588823246522), (1164, 0.8534471875172311), (1224, 0.8527817705498422), (1175, 0.8524823476721942), (1225, 0.8515948796308773), (1214, 0.8510218766257365), (1198, 0.8508692848775299), (1219, 0.8505166086990577), (1166, 0.8504925399132279), (1187, 0.850195634069446), (1211, 0.8483656437876

In [29]:
# print suggested movies
i = 1

for song in sorted_similar_songs:
  index = song[0]
  title_of_movie = merged_df[merged_df.index == index]['track_name'].values[0]
  artist_name = merged_df[merged_df.index == index]['track_artist'].values[0]
  if (i < 30) :
    print(i, " ", title_of_movie, " ", artist_name)
    i += 1


1   Shape of You   Ed Sheeran
2   Story of My Life   One Direction
3   Hold Up   Beyoncé
4   Run the World Girls   Beyoncé
5   Set Fire to the Rain   Adele
6   Impossible   James Arthur
7   Liar   Camila Cabello
8   Pumped Up Kicks   Foster The People
9   Work from Home feat. Ty Dolla $ign   Fifth Harmony
10   New Rules   Dua Lipa
11   Uptown Funk   Mark Ronson
12   Don't Let Me Down   The Chainsmokers
13   All About That Bass   Meghan Trainor
14   New Light   John Mayer
15   All of Me   John Legend
16   Rude   MAGIC!
17   Sunday Best   Surfaces
18   We Can't Stop   Miley Cyrus
19   Love Lies with Normani   Khalid
20   Say You Won't Let Go   James Arthur
21   PILLOWTALK   ZAYN
22   Dusk Till Dawn - Radio Edit   ZAYN
23   Cry for Me   Camila Cabello
24   Cheap Thrills   Sia
25   Payphone   Maroon 5
26   Sweet Nothing feat. Florence Welch   Calvin Harris
27   Chandelier   Sia
28   What About Us   P!nk
29   Ex's & Oh's   Elle King
